# MIDS - w261 Machine Learning At Scale
__Course Lead:__ Dr James G. Shanahan (__email__ Jimi via  James.Shanahan _AT_ gmail.com)

## Final Exam Exercise

---
__Name:__   Megan Jasek  <br \> 
__Class:__ MIDS w261 (Summer 2016, Section ?) <br \> 
__Email:__ meganjasek@ischool.berkeley.edu <br \> 
__Week:__  14 <br \>

In [ ]:
# purpose of cell: download and view test data set from DropBox for use in subsequent cells

#!wget https://www.dropbox.com/sh/2c0k5adwz36lkcw/AADxzBgNxNF5Q6-eanjnK64qa/PageRank-test.txt
#!cat PageRank-test.txt

# *Spark* implementation of basic PageRank

-----

- Per Jimi, if we translate the MapReduce concepts to Spark, that will start our juices and we should be set up nicely for the Final Exam.
- We can run locally since the Final Exam is expected to be similar in format to the Midterm Exam (e.g., not require AWS or SoftLayer).
- He encouraged us to feel free to share notebook(s) on Google Groups since that might help each other.

-----

**The remaining text below is verbatim from HW9.1, except for the first sentence which replaces 'MRJob' with 'Spark'.**

As we had written for HW9.1 (basic MRJob implementation), now write a basic Spark implementation of the iterative PageRank algorithm that takes sparse adjacency lists as input.

Make sure that your implementation utilizes teleportation (1-damping/the number of nodes in the network), and further, distributes the mass of dangling nodes with each iteration so that the output of each iteration is correctly normalized (sums to 1).

[NOTE: The PageRank algorithm assumes that a random surfer (walker), starting from a random web page, chooses the next page to which it will move by clicking at random, with probability d, one of the hyperlinks in the current page. This probability is represented by a so-called ‘damping factor’ d, where d ∈ (0, 1). Otherwise, with probability (1 − d), the surfer jumps to any web page in the network. If a page is a dangling end, meaning it has no outgoing hyperlinks, the random surfer selects an arbitrary web page from a uniform distribution and “teleports” to that page.

As you build your code, use the test data
s3://ucb-mids-mls-networks/PageRank-test.txt

Or under the Data Subfolder for HW7 on Dropbox with the same file name. 
(On Dropbox https://www.dropbox.com/sh/2c0k5adwz36lkcw/AAAAKsjQfF9uHfv-X9mCqr9wa?dl=0)

with teleportation parameter set to 0.15 (1-d, where d, the damping factor is set to 0.85), and crosscheck
your work with the true result, displayed in the first image in the Wikipedia article: https://en.wikipedia.org/wiki/PageRank

and here for reference are the corresponding PageRank probabilities:

A,0.033 <br />
B,0.384 <br />
C,0.343 <br />
D,0.039 <br />
E,0.081 <br />
F,0.039 <br />
G,0.016 <br />
H,0.016 <br />
I,0.016 <br />
J,0.016 <br />
K,0.016 <br />

-----

### Create a Spark Context to use throughout this homework

In [1]:
import pyspark
from pyspark.sql import SQLContext

# We can give a name to our app (to find it in Spark WebUI) and configure execution mode
# In this case, it is local multicore execution with "local[*]"
app_name = "HW11"
master = "local[*]"
conf = pyspark.SparkConf().setAppName(app_name).setMaster(master)
sc.stop()
sc = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(sc)

print sc
print sqlContext

### Read the data into an RDD and cache it

In [2]:
!cat PageRank-test.txt

B	{'C': 1}
C	{'B': 1}
D	{'A': 1, 'B': 1}
E	{'D': 1, 'B': 1, 'F': 1}
F	{'B': 1, 'E': 1}
G	{'B': 1, 'E': 1}
H	{'B': 1, 'E': 1}
I	{'B': 1, 'E': 1}
J	{'E': 1}
K	{'E': 1}

In [3]:
import ast
import numpy as np

# return a node and adjacency list for all nodes in the file plus all nodes
# in the adjacency lists of nodes in the file.  For the latter, output (node, {})
def parsePoint(line):
    all_nodes = []
    node_num, adj_dict = line.strip().split('\t')
    adj_dict = ast.literal_eval(adj_dict)
    all_nodes.append((str(node_num), adj_dict))
    for node in adj_dict:
        all_nodes.append((node, {}))
    return all_nodes

# aggregate all adjacency lists by key
def reduceNodes(x, y):
    if x != {}:
        return x
    elif y != {}:
        return y
    else:
        return {}
    
# Initialize all nodes with a page rank of 1/n
def initNode(node, n):
    return (node[0], (node[1], 1.0/n))

# Distribute the page rank across all nodes and keep track of dangling mass page rank with '*'
def mapPageRankStep1(node):
    name = node[0]
    adj_list = node[1][0]
    page_rank = node[1][1]
    all_nodes = []
    degree = float(len(adj_list))
    if adj_list == {}:
        all_nodes.append(('*',({}, page_rank)))
    all_nodes.append((name, (adj_list, 0.0)))
    for item in adj_list:
        all_nodes.append((item, ({}, page_rank/degree)))
    return all_nodes

# aggregate the page rank
def reducePageRankStep1(x, y):
    page_rank = x[1]+y[1]
    if x[0] != {}:
        return (x[0], page_rank)
    elif y[0] != {}:
        return (y[0], page_rank)
    else:
        return ({}, page_rank)

# Return the final page rank calculation
def mapPageRankStep2(node, damping_factor, total_nodes, dangling_mass):
    page_rank = (1-damping_factor)*(1.0/total_nodes)+damping_factor*((dangling_mass/total_nodes)+node[1][1])
    return (node[0], (node[1][0], page_rank))

# Calculate the error between two node RDDs
def getError(n1, n2):
    return sum(abs(np.asarray(n1.map(lambda x: x[1][1]).collect()) - np.asarray(n2.map(lambda x: x[1][1]).collect())))

In [4]:
# Set parameters
damping_factor = 0.85
epsilon = 0.001

# Read in data and count nodes.  Be sure and entries for the dangling nodes.
fileName = 'PageRank-test.txt'
rawNodes = sc.textFile(fileName).flatMap(parsePoint).reduceByKey(reduceNodes)
total_nodes = rawNodes.count()
print 'Total nodes', total_nodes

# Initialize Page Ranks
nodesInit = rawNodes.map(lambda node: initNode(node, total_nodes)).cache()
print 'Initial page ranks'
#print nodesInit.take(total_nodes)
for x in nodesInit.sortByKey().collect():
    print x[0], x[1][1]
print

nodesPrev = nodesInit
i = 1
stop = False
while (stop == False):
    print 'Iteration', str(i) 
    # Loop through all of the nodes and accumulate the page rank.  Keep a special node called '*'
    # to store the dangling mass
    nodes1 = nodesPrev.flatMap(mapPageRankStep1).reduceByKey(reducePageRankStep1).cache()
    # Extract the dangling mass from the RDD
    dangling_mass = nodes1.filter(lambda node: node[0]=='*').collect()[0][1][1]
    print 'Dangling mass', dangling_mass

    # Use the dangling mass to calculate the final page ranks for this iteration
    nodes2 = (nodes1.filter(lambda node: node[0]!='*')
              .map(lambda node: mapPageRankStep2(node, damping_factor, total_nodes, dangling_mass))
              .cache())
    print 'Total page rank', nodes2.map(lambda x: x[1][1]).reduce(lambda x,y: x+y)
    # Calculate the error between page ranks in this iteration and the previous iteration
    error = getError(nodesPrev, nodes2)
    print 'Error', error
    # If the error is less than epsilon, then set stop to True
    stop = error < epsilon
    nodesPrev = nodes2
    i += 1
    print
# Print the final values
print 'Final page ranks'
for x in nodes2.sortByKey().collect():
    print x[0], x[1][1]

Total nodes 11
Initial page ranks
A 0.0909090909091
B 0.0909090909091
C 0.0909090909091
D 0.0909090909091
E 0.0909090909091
F 0.0909090909091
G 0.0909090909091
H 0.0909090909091
I 0.0909090909091
J 0.0909090909091
K 0.0909090909091

Iteration 1
Dangling mass 0.0909090909091
Total page rank 1.0
Error 0.943663911846

Iteration 2
Dangling mass 0.0592975206612
Total page rank 1.0
Error 0.640171550213

Iteration 3
Dangling mass 0.0379464062109
Total page rank 1.0
Error 0.382958337128

Iteration 4
Dangling mass 0.0640190695934
Total page rank 1.0
Error 0.302769848196

Iteration 5
Dangling mass 0.0375959647951
Total page rank 1.0
Error 0.227349178989

Iteration 6
Dangling mass 0.0386749363905
Total page rank 1.0
Error 0.18873867623

Iteration 7
Dangling mass 0.0341177257382
Total page rank 1.0
Error 0.152290457013

Iteration 8
Dangling mass 0.0346526855821
Total page rank 1.0
Error 0.128755161228

Iteration 9
Dangling mass 0.0332641479909
Total page rank 1.0
Error 0.107417609557

Iteration 10